## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [2]:
# 設定 data_path
dir_data = './data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [3]:
name = 'DAYS_BIRTH'

In [4]:
# 等寬劃分
app_train["equal_width_" + name] = pd.cut(app_train[name], 4)
app_train["equal_width_" + name].value_counts() # 每個 bin 的值的範圍大小都是一樣的

(11924.0, 16359.0]    100808
(16359.0, 20794.0]     84685
(7471.26, 11924.0]     66536
(20794.0, 25229.0]     55482
Name: equal_width_DAYS_BIRTH, dtype: int64

In [5]:
app_train["equal_freq_" + name] = pd.qcut(app_train[name], 4)
app_train["equal_freq_" + name].value_counts() # 每個 bin 的資料筆數是一樣的

(15750.0, 19682.0]     76887
(7488.999, 12413.0]    76884
(12413.0, 15750.0]     76877
(19682.0, 25229.0]     76863
Name: equal_freq_DAYS_BIRTH, dtype: int64

In [6]:
col_min = app_train[name].min()
col_max = app_train[name].max()
col_diff = col_max - col_min
app_train["customized_grp_" + name] = pd.cut(app_train[name], [col_min, col_min + 0.1 * col_diff, col_min + 0.2 * col_diff, col_min + 0.5 * col_diff, col_max])
app_train["customized_grp_" + name].value_counts()

(16359.0, 25229.0]    140167
(11037.0, 16359.0]    120538
(9263.0, 11037.0]      33120
(7489.0, 9263.0]       13685
Name: customized_grp_DAYS_BIRTH, dtype: int64

In [7]:
app_train[[name, "equal_width_" + name, "equal_freq_" + name, "customized_grp_" + name]].head()

,DAYS_BIRTH,equal_width_DAYS_BIRTH,equal_freq_DAYS_BIRTH,customized_grp_DAYS_BIRTH
0,9461,"(7471.26, 11924.0]","(7488.999, 12413.0]","(9263.0, 11037.0]"
1,16765,"(16359.0, 20794.0]","(15750.0, 19682.0]","(16359.0, 25229.0]"
2,19046,"(16359.0, 20794.0]","(15750.0, 19682.0]","(16359.0, 25229.0]"
3,19005,"(16359.0, 20794.0]","(15750.0, 19682.0]","(16359.0, 25229.0]"
4,19932,"(16359.0, 20794.0]","(19682.0, 25229.0]","(16359.0, 25229.0]"
